In [1]:
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
# from imblearn.ensemble import EasyEnsembleClassifier

In [2]:
df=pd.read_excel('cleaned_accident_data.xlsx')
df.head()

,Accident_months,Day_of_Week,Junction_Control,Junction_Detail,Light_Conditions,Local_Authority_District,Carriageway_Hazards,Number_of_Casualties,Number_of_Vehicles,Police_Force,Road_Surface_Conditions,Road_Type,Speed_limit,Urban_or_Rural_Area,Weather_Conditions,Vehicle_Type,Accident_Severity
0,6,Saturday,Data missing or out of range,Not at junction or within 20 metres,Daylight,North Larkshire,None,1,2,Strathclyde,Dry,Single carriageway,60,Rural,Fine no high winds,Car,Fatal
1,7,Saturday,Data missing or out of range,Not at junction or within 20 metres,Darkness - no lighting,North Larkshire,Previous accident,1,6,Strathclyde,Dry,Dual carriageway,70,Rural,Fine no high winds,Motorcycle 125cc and under,Fatal
2,8,Thursday,Data missing or out of range,Not at junction or within 20 metres,Daylight,North Larkshire,None,2,2,Strathclyde,Dry,Single carriageway,60,Rural,Fine no high winds,Car,Fatal
3,1,Friday,Give way or uncontrolled,T or staggered junction,Daylight,Eden,None,1,1,Cumbria,Wet or damp,Single carriageway,30,Rural,Fine no high winds,Car,Slight
4,1,Monday,Data missing or out of range,Not at junction or within 20 metres,Daylight,Copeland,None,2,2,Cumbria,Frost or ice,Single carriageway,60,Rural,Fine no high winds,Car,Slight


In [3]:
# Encode categorical columns
categorical_cols = ['Day_of_Week', 'Junction_Control', 'Junction_Detail', 'Light_Conditions', 
                    'Carriageway_Hazards', 'Number_of_Casualties', 'Number_of_Vehicles','Police_Force',
                    'Road_Surface_Conditions', 'Road_Type', 'Urban_or_Rural_Area', 
                    'Weather_Conditions', 'Vehicle_Type', 'Accident_Severity']

In [4]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
for i in categorical_cols:
    df[i]=le.fit_transform(df[i])
df

,Accident_months,Day_of_Week,Junction_Control,Junction_Detail,Light_Conditions,Local_Authority_District,Carriageway_Hazards,Number_of_Casualties,Number_of_Vehicles,Police_Force,Road_Surface_Conditions,Road_Type,Speed_limit,Urban_or_Rural_Area,Weather_Conditions,Vehicle_Type,Accident_Severity
0,6,2,2,3,4,North Larkshire,1,0,1,40,0,3,60,0,1,2,0
1,7,2,2,3,3,North Larkshire,4,0,5,40,0,0,70,0,1,6,0
2,8,4,2,3,4,North Larkshire,1,1,1,40,0,3,60,0,1,2,0
3,1,0,3,8,4,Eden,1,0,0,7,4,3,30,0,1,2,2
4,1,1,2,3,4,Copeland,1,1,1,7,2,3,60,0,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30424,3,3,2,3,4,Greenwich,1,0,1,29,0,3,30,1,1,2,0
30425,3,3,2,3,4,Lambeth,1,0,0,29,0,3,30,1,1,6,0
30426,10,0,2,3,4,Enfield,1,0,0,29,0,3,30,1,1,9,0
30427,1,5,3,7,4,Hounslow,1,0,0,29,4,0,50,1,1,2,0


In [5]:
# target encode
# import category_encoders as ce
# target_encoder = ce.TargetEncoder(cols=['Police_Force'], smoothing=10)
# df['Police_Force_encoded'] = target_encoder.fit_transform(df['Police_Force'], df['Accident_Severity'])


In [6]:
df.head()

,Accident_months,Day_of_Week,Junction_Control,Junction_Detail,Light_Conditions,Local_Authority_District,Carriageway_Hazards,Number_of_Casualties,Number_of_Vehicles,Police_Force,Road_Surface_Conditions,Road_Type,Speed_limit,Urban_or_Rural_Area,Weather_Conditions,Vehicle_Type,Accident_Severity
0,6,2,2,3,4,North Larkshire,1,0,1,40,0,3,60,0,1,2,0
1,7,2,2,3,3,North Larkshire,4,0,5,40,0,0,70,0,1,6,0
2,8,4,2,3,4,North Larkshire,1,1,1,40,0,3,60,0,1,2,0
3,1,0,3,8,4,Eden,1,0,0,7,4,3,30,0,1,2,2
4,1,1,2,3,4,Copeland,1,1,1,7,2,3,60,0,1,2,2


In [7]:
# df.Police_Force_encoded.value_counts().count()

In [8]:
df.shape

(30429, 17)

In [9]:
df.corr()

,Accident_months,Day_of_Week,Junction_Control,Junction_Detail,Light_Conditions,Carriageway_Hazards,Number_of_Casualties,Number_of_Vehicles,Police_Force,Road_Surface_Conditions,Road_Type,Speed_limit,Urban_or_Rural_Area,Weather_Conditions,Vehicle_Type,Accident_Severity
Accident_months,1.000000,0.006211,0.012245,0.006685,-0.059998,0.000347,-0.011916,-0.002624,0.003039,0.106697,0.012193,-0.021139,0.019750,0.104748,0.001441,0.057750
Day_of_Week,0.006211,1.000000,-0.003588,0.003087,0.032189,-0.004806,-0.017725,0.009203,-0.000097,0.014994,-0.007397,-0.014257,0.015113,0.005625,-0.001251,0.021014
Junction_Control,0.012245,-0.003588,1.000000,0.396612,0.033025,-0.002044,0.004366,0.021201,-0.106778,0.024165,0.128563,0.018915,-0.060981,0.008081,0.011908,-0.064711
Junction_Detail,0.006685,0.003087,0.396612,1.000000,0.016030,-0.016762,-0.061384,-0.004477,0.049654,-0.016666,0.090756,-0.124784,0.102254,-0.008256,-0.000661,0.073348
Light_Conditions,-0.059998,0.032189,0.033025,0.016030,1.000000,0.005130,-0.034723,0.041098,-0.058162,-0.172892,0.025424,0.064477,-0.073610,-0.142949,0.003325,0.029484
Carriageway_Hazards,0.000347,-0.004806,-0.002044,-0.016762,0.005130,1.000000,0.012348,0.027850,-0.031853,0.016697,-0.049641,0.070228,-0.051475,0.008466,0.006647,-0.029931
Number_of_Casualties,-0.011916,-0.017725,0.004366,-0.061384,-0.034723,0.012348,1.000000,0.217535,-0.057922,0.063737,-0.050692,0.224444,-0.211919,0.028679,0.008040,-0.190875
Number_of_Vehicles,-0.002624,0.009203,0.021201,-0.004477,0.041098,0.027850,0.217535,1.000000,0.007100,-0.001399,-0.072252,0.066288,-0.032652,0.001209,-0.000206,0.078738
Police_Force,0.003039,-0.000097,-0.106778,0.049654,-0.058162,-0.031853,-0.057922,0.007100,1.000000,-0.072497,-0.016517,-0.237286,0.322737,-0.037534,-0.015368,0.044037
Road_Surface_Conditions,0.106697,0.014994,0.024165,-0.016666,-0.172892,0.016697,0.063737,-0.001399,-0.072497,1.000000,-0.004071,0.093329,-0.104443,0.668534,-0.002619,-0.051547


In [10]:
import pandas as pd
import numpy as np

# Correlation matrix
correlation_matrix = df.corr()

# Threshold for high correlation
threshold = 0.7

# Find highly correlated features
high_corr_pairs = [(i, j) for i in correlation_matrix.columns for j in correlation_matrix.columns
                   if i != j and abs(correlation_matrix.loc[i, j]) > threshold]

print("Highly correlated pairs:")
for pair in high_corr_pairs:
    print(pair, correlation_matrix.loc[pair[0], pair[1]])

# Decide which features to drop
# Example: Keep the first feature in each pair
features_to_drop = [pair[1] for pair in high_corr_pairs]
features_to_drop = list(set(features_to_drop))  # Remove duplicates

# Drop features from dataset
# data_reduced = df.drop(columns=features_to_drop)
# data_reduced.shape

Highly correlated pairs:
('Speed_limit', 'Urban_or_Rural_Area') -0.7241079843749418
('Urban_or_Rural_Area', 'Speed_limit') -0.7241079843749418


In [11]:
# Drop redundant or highly correlated features
df.drop(['Urban_or_Rural_Area', 'Local_Authority_District'], axis=1, inplace=True)


In [12]:
df.head(1)

,Accident_months,Day_of_Week,Junction_Control,Junction_Detail,Light_Conditions,Carriageway_Hazards,Number_of_Casualties,Number_of_Vehicles,Police_Force,Road_Surface_Conditions,Road_Type,Speed_limit,Weather_Conditions,Vehicle_Type,Accident_Severity
0,6,2,2,3,4,1,0,1,40,0,3,60,1,2,0


In [13]:
# Separate features and target
X = df.drop(columns=['Accident_Severity'])
y = df['Accident_Severity']
X

,Accident_months,Day_of_Week,Junction_Control,Junction_Detail,Light_Conditions,Carriageway_Hazards,Number_of_Casualties,Number_of_Vehicles,Police_Force,Road_Surface_Conditions,Road_Type,Speed_limit,Weather_Conditions,Vehicle_Type
0,6,2,2,3,4,1,0,1,40,0,3,60,1,2
1,7,2,2,3,3,4,0,5,40,0,0,70,1,6
2,8,4,2,3,4,1,1,1,40,0,3,60,1,2
3,1,0,3,8,4,1,0,0,7,4,3,30,1,2
4,1,1,2,3,4,1,1,1,7,2,3,60,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30424,3,3,2,3,4,1,0,1,29,0,3,30,1,2
30425,3,3,2,3,4,1,0,0,29,0,3,30,1,6
30426,10,0,2,3,4,1,0,0,29,0,3,30,1,9
30427,1,5,3,7,4,1,0,0,29,4,0,50,1,2


In [14]:
y

0        0
1        0
2        0
3        2
4        2
        ..
30424    0
30425    0
30426    0
30427    0
30428    0
Name: Accident_Severity, Length: 30429, dtype: int32

In [15]:
from sklearn.preprocessing import StandardScaler
# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [16]:
X_scaled

array([[-0.19699979, -0.48133517, -0.49805865, ...,  2.66605087,
        -0.38842024, -0.36160397],
       [ 0.09890832, -0.48133517, -0.49805865, ...,  3.68470057,
        -0.38842024,  1.09785781],
       [ 0.39481644,  0.48892404, -0.49805865, ...,  2.66605087,
        -0.38842024, -0.36160397],
       ...,
       [ 0.98663266, -1.45159439, -0.49805865, ..., -0.38989822,
        -0.38842024,  2.19245415],
       [-1.67654034,  0.97405365,  0.74674025, ...,  1.64740117,
        -0.38842024, -0.36160397],
       [-1.67654034, -0.48133517,  0.74674025, ..., -0.38989822,
        -0.38842024, -0.36160397]])

In [17]:
from imblearn.over_sampling import SMOTE
# Oversample minority class
X_resample, Y_resample = SMOTE(random_state=42).fit_resample(X_scaled, y)

In [18]:
X_resample

array([[-0.19699979, -0.48133517, -0.49805865, ...,  2.66605087,
        -0.38842024, -0.36160397],
       [ 0.09890832, -0.48133517, -0.49805865, ...,  3.68470057,
        -0.38842024,  1.09785781],
       [ 0.39481644,  0.48892404, -0.49805865, ...,  2.66605087,
        -0.38842024, -0.36160397],
       ...,
       [ 0.14947613, -1.45159439,  0.74674025, ..., -0.38989822,
        -0.38842024, -0.36160397],
       [ 1.10050108,  1.2725005 ,  0.74674025, ..., -0.38989822,
        -0.38842024, -0.36160397],
       [ 0.98663266, -1.45159439,  0.74674025, ..., -0.38989822,
        -0.38842024, -0.36160397]])

In [19]:
Y_resample

0        0
1        0
2        0
3        2
4        2
        ..
66442    1
66443    1
66444    1
66445    1
66446    1
Name: Accident_Severity, Length: 66447, dtype: int32

In [20]:
from sklearn.model_selection import cross_val_score,KFold
kfold=KFold(n_splits=10) #Use StratifiedKFold for better class balance in cross-validation
model1=XGBClassifier()
result=cross_val_score(model1,X_resample,Y_resample,cv=kfold)

In [21]:
result

array([0.68306998, 0.89751693, 0.80993228, 0.85342363, 0.83491347,
       0.82994733, 0.83656885, 0.85881999, 0.76595424, 0.76836243])

In [22]:
result.mean()

0.8138509125156457

In [23]:
# Cross-validation setup
models = [
    ('Decision Tree', DecisionTreeClassifier()),
    ('Random Forest', RandomForestClassifier(max_depth=10, n_estimators=100, min_samples_split=5, random_state=42)),
    ('Naive Bayes', GaussianNB()),
    ('XGBoost', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)),
    ('Bagging Classifier', BaggingClassifier())
]

In [24]:
# StratifiedKFold for imbalanced data
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [25]:
# Evaluate models
results = {}
for name, model in models:
    scores = cross_val_score(model, X_resample, Y_resample, cv=cv, scoring='accuracy')  # Use F1-macro for imbalanced data
    results[name] = scores
    print(f"{name}: Mean accuracy = {scores.mean():.4f}, Std = {scores.std():.4f}")


Decision Tree: Mean accuracy = 0.8220, Std = 0.0031
Random Forest: Mean accuracy = 0.7317, Std = 0.0044
Naive Bayes: Mean accuracy = 0.5146, Std = 0.0048
XGBoost: Mean accuracy = 0.8589, Std = 0.0041
Bagging Classifier: Mean accuracy = 0.8507, Std = 0.0032


In [26]:
# Identify the best model
best_model_name = max(results, key=lambda k: results[k].mean())
print(f"\nBest Model: {best_model_name} with accuracy = {results[best_model_name].mean():.4f}")



Best Model: XGBoost with accuracy = 0.8589
